In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import pickle
import math
from sklearn import preprocessing

In [23]:
def supply():
    df = pd.read_csv('raw_data/sales_supply.csv', header=0, delimiter=',')
    area = df['Area'].tolist()
    df = df.drop('Area',1)
    df = df.drop('1/1/17',1) #drop data since 2017
    df = df.drop('2/1/17',1)
    for year in ['15', '16']:
        for month in list(range(1, 13)):
            df = df.drop(str(month)+'/1/'+year, 1)

    column = df.columns 
    #normalization
    np=df.values

    normalized=[]
    for row in np:
        normalized.append(((row-min(row))/(max(row)-min(row)))+1)

    #create support dicts
    df_normal=pd.DataFrame(data=normalized, columns=column)

    supply ={}
    for year in list(range(2009, 2015)):
        supply[year]={}
        for nei in area:
            supply[year][nei]={}
            for month in list(range(1, 13)):
                supply[year][nei][month]={}

    for col in column:
        tmp_month ={}
        col_tmp = df_normal[col].tolist()
        year=int(col.split('/')[2])+2000
        month=int(col.split('/')[0])
        tmp_month={}
        for r_idx, ele in enumerate(col_tmp):
            if not ele:
                tmp_month['supply']= 0
            else:
                tmp_month['supply']= ele
            supply[year][area[r_idx]][month]=tmp_month

    with open('p_file/supply.p','wb') as f:
        pickle.dump(supply, f)

In [24]:
supply()

In [2]:
#create data_id dict
def data_id(): 
    with open('p_file/se2sba.p','rb') as f:
        trans = pickle.load(f)
    
    with open('p_file/supply.p','rb') as f:
        supply = pickle.load(f)
        
    with open('p_file/f1.p','rb') as f:
        f1 = pickle.load(f)
    
    with open('p_file/f2.p','rb') as f:
        f2 = pickle.load(f)

    target = pd.read_csv('raw_data/target variable matrix_1.csv', header=0, delimiter=',')
    area = target['Area'].tolist()
    target = target.drop('Area', 1)

    column = target.columns

    #create dict
    data_id ={}
    for nei in area:
        data_id[nei]={}
        for year in list(range(2009, 2015)):
            data_id[nei][year]={}
            data_id[nei]['sba']=trans[nei]
            data_id[nei]['f1']=f1[trans[nei]]
            data_id[nei]['f2']=f2[trans[nei]]
            for month in list(range(1, 13)):
                data_id[nei][year][month]={}

    for col in column:
        col_tmp = target[col].tolist()
        year=int(col.split('/')[2])+2000
        month=int(col.split('/')[0])
        for r_idx, ele in enumerate(col_tmp):
            tmp_month={}
            if not ele:
                tmp_month['index']= 0
            else:
                tmp_month['index']= ele
            try:
                tmp_month['supply']=supply[year][area[r_idx]][month]['supply']
            except:
                tmp_month['supply']=0
            data_id[area[r_idx]][year][month]=tmp_month


    # save dict
    with open("p_file/data_id.p", "wb") as f:
        pickle.dump(data_id, f)
    with open('p_file/neighbor.p','wb') as f:
        pickle.dump(area, f)

In [5]:
data_id()

In [22]:
local_pop= pd.read_csv('raw_data/sub-borougharea-borninnewyorkstate.csv',header=0, delimiter=',')
foreign_pop= pd.read_csv('raw_data/sub-borougharea-foreign-bornpopulation.csv',header=0, delimiter=',')
travel_time= pd.read_csv('raw_data/sub-borougharea-meantraveltimetoworkminutes.csv',header=0, delimiter=',')
income= pd.read_csv('raw_data/sub-borougharea-medianhouseholdincome2015.csv',header=0, delimiter=',')
loan= pd.read_csv('raw_data/sub-borougharea-refinanceloanrateper1000properties.csv',header=0, delimiter=',')
vacancy= pd.read_csv('raw_data/sub-borougharea-rentalvacancyrate.csv',header=0, delimiter=',')

#sort by area
local_pop=local_pop.sort_values(['Sub-Borough Area'])
foreign_pop=foreign_pop.sort_values(['Sub-Borough Area'])
travel_time=travel_time.sort_values(['Sub-Borough Area'])
income=income.sort_values(['Sub-Borough Area'])
loan=loan.sort_values(['Sub-Borough Area'])
vacancy=vacancy.sort_values(['Sub-Borough Area'])

area = local_pop['Sub-Borough Area'].tolist()

local_pop=local_pop.drop('Sub-Borough Area', 1).values
foreign_pop=foreign_pop.drop('Sub-Borough Area', 1).values
travel_time=travel_time.drop('Sub-Borough Area', 1).values
income=income.drop('Sub-Borough Area', 1).values
loan=loan.drop('Sub-Borough Area', 1).values
vacancy=vacancy.drop('Sub-Borough Area', 1).values

furman={}
for i in range(2009,2015):
    tmp_year={}
    for r_idx, a in enumerate(area):
        if np.isnan(local_pop[r_idx, i-2009]):
            local_pop_ele=0
        else:
            local_pop_ele=local_pop[r_idx, i-2009]
        if np.isnan(foreign_pop[r_idx, i-2009]):
            foreign_pop_ele=0
        else:
            foreign_pop_ele=foreign_pop[r_idx, i-2009]
        if np.isnan(income[r_idx, i-2009]):
            income_ele=0
        else:
            income_ele=income[r_idx, i-2009]
        if np.isnan(travel_time[r_idx, i-2009]):
            travel_time_ele=0
        else:
            travel_time_ele=travel_time[r_idx, i-2009]
        if np.isnan(loan[r_idx, i-2009]):
            loan_ele=0
        else:
            loan_ele=loan[r_idx, i-2009]
        if np.isnan(vacancy[r_idx, i-2009]):
            vacancy_ele=0
        else:
            vacancy_ele=vacancy[r_idx, i-2009]
        
        tmp_year[a]={'local_pop':local_pop_ele, 'foreign_pop':foreign_pop_ele,
                     'income':income_ele, 'travel_time':travel_time_ele,
                     'loan':loan_ele, 'vacancy':vacancy_ele}
    furman[i]=tmp_year
    
with open('p_file/furman.p','wb') as f:
    pickle.dump(furman, f)

In [12]:
park=pd.read_csv('raw_data/communitydistrict-residentialunitswithin14mileofapark.csv', delimiter=',')
sub=pd.read_csv('raw_data/communitydistrict-residentialunitswithin12mileofasubwaystation.csv', delimiter=',')
crime=pd.read_csv('raw_data/communitydistrict-seriouscrimerateper1000residents.csv')

park=park.sort_values(['area'])
sub=sub.sort_values(['area'])
crime=crime.sort_values(['area'])
crime=crime.drop('area',1)
column_crime=crime.columns

area = park['area'].tolist()
area1 =[]

furman2={}

for idx, ele in enumerate(area):
    a=ele.split(' ')[0]+ele.split(' ')[1]
    area1.append(a)
    furman2[a]={'park':park['index'][idx], 'sub':sub['index'][idx]}
    for i in range(2009, 2015):
        furman2[a][i]={}
        

for col in column_crime:
    c = crime[col].tolist()
    year=int(col)
    for r_idx, ele in enumerate(c):
        furman2[area1[r_idx]][year]=ele
    
with open('p_file/furman2.p','wb') as f:
    pickle.dump(furman2, f)